In [ ]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
import pandas as pd
octave.addpath('C:/dynare/6.1/matlab')  
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

In [ ]:
n_goods=3

In [ ]:
#from MyPy import DyPy

In [ ]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=10000, irf=100);")

In [ ]:
%run make_model.ipynb
%run make_vars.ipynb
%run make_parameters_and_shocks.ipynb
%run make_steady_state.ipynb


Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [ ]:
octave.dynare('model_1.mod')
oo = octave.pull('oo_')
mm = octave.pull('M_')

In [ ]:
names1 = [attr for attr in dir(oo) if not attr.startswith('__')]

Now we want to create a class where each each attribute in oo becomes an attribute of this class. So far we take only one item from mm. This is a list of endogenous variables. Below we define the class. 

In [ ]:
class from_octave:
	def __init__(self, oo_structure, mm_structure, attribute_names):
		self.oo_structure = oo_structure
		self.mm_structure = mm_structure
		self. attribute_names =  attribute_names
		self.create_attributes()
		self.make_list()
     
	def create_attributes(self): 
		for attr in self.attribute_names:
			setattr(self, attr, getattr(self.oo_structure, attr)) #this is the attributes we pull from the oo structure. We are making them attributes of the class
	
	def make_list(self): 
		self.names = list(getattr(self.mm_structure, 'endo_names')) #getting one item from the mm structure. This is the endo_names
		self.names = [item[0] for item in self.names] #pulling mm gives us a list of cell arrays. We are pulling the first item from each cell array

We see that we now can reference any of the data from `oo` with the `tt.` notation. Some examples are below. 

In [ ]:
tt = from_octave(oo, mm, names1)

In [ ]:
tt.var

In [ ]:
tt.steady_state

In [ ]:
tt.irfs

In [ ]:
tt.names

We will care about the irfs but these are also of type `oct2py.io.Struct`. We convert it as above. We will likley make this part of the above class later, but for now we create a new class. 

In [ ]:
names2 = [attr for attr in dir(tt.irfs) if attr.endswith('shock')]
#names2 = [x for _, x in sorted(zip(tt.names, names2), key=lambda pair: pair[0][0])]

I will put more desciptors here later. However, it is getting the impulse responses as difference and levels. It is simultanosuly creating two dataframes containing all differences and levels data.  

In [ ]:
#recall make_levels is created in simpler ffolder
import numpy as np
import pandas as np
class make_irfs:
	def __init__(self, dta, vars, names):
		self.data = dta
		self.vars = vars
		self.names = names
		self.create_difs()
		
  	
	def create_difs(self):
		for var in self.vars:
			setattr(self, var, pd.DataFrame())  # Initialize an empty DataFrame with the dynamic name
			list_short = [name for name in self.names if name.startswith(var)]
			for attr in list_short:
				x = getattr(self.data, attr)
				x = x.flatten()
				getattr(self, var)[attr] = x  # Assign the flattened array to the DataFrame
				new_columns = [col.replace(var + '_', '').replace('shock', '') for col in getattr(self, var).columns]
				getattr(self, var).columns = new_columns
    
    



In [ ]:
varlist=['c','i','y','k','n0','s0']

In [ ]:
ir = make_irfs(tt.irfs,varlist, names2)
print(ir.__dict__.keys())

Lets get a sense of what we have. 

In [ ]:
ir.y

This will also need more documentation, but it is getting the results of dynare's simulation. It defines the individual series and also puts all in a dataframe. 

In [ ]:
import numpy as np

class make_data:
	def __init__(self, dta,names):
		self.data = dta
		self.names = names
		self.make_levels()
		self.make_raw()

	def make_raw(self):
		self.df_raw = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			new_attr =attr + 'r'
			raw_level = self.data[j]
			setattr(self, new_attr, raw_level)
			self.df_raw[new_attr] = raw_level
			
  		
	def make_levels(self):
		self.df = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			raw_level = self.data[j]
			if attr != 'i' and 'theta' not in attr:
				current_level=np.exp(raw_level)
			else:
				current_level=raw_level
			setattr(self, attr, current_level)
			self.df[attr] = current_level  # Store in DataFrame



In [ ]:
print(tt.endo_simul)

In [ ]:
eg=make_data(tt.endo_simul,tt.names)

In [ ]:
eg.df

In [ ]:
params.keys()

In [ ]:
def x_find(g,t):
    thetag='theta'+str(g)
    ag='a'+str(g)
    omegag='omega'+str(g)
    thetai=eg.df[thetag][t]
    theta0=eg.df['theta0'][t]
    ai=eg.df[ag][t]
    a0=eg.df['a0'][t]
    k=eg.df['k'][t-1]
    s0=eg.df['s0'][t]
    n0=eg.df['n0'][t]
    theta0=eg.df['theta0'][t]
    eps=params['eps']
    omegai=params[omegag]
    omega0=params['omega0']
    
    x=(k*s0/n0)**((theta0-thetai)*(1-eps))*(omegai/omega0)**eps* \
    ((a0)**(1-theta0)/(ai)**(1-thetai)*(theta0/thetai)**thetai*((1-theta0)/(1-thetai))**(1-thetai))**(1-eps)
    
    return x

In [ ]:
def X_find(t):
    X=0
    for g in range(0,n_goods):
        X+=x_find(g,t)
    return X

In [ ]:
def Z_find(t):
    X=0
    for g in range(0,n_goods):
        thetag='theta'+str(g)
        thetai=eg.df[thetag][t]
        X+=x_find(g,t)*thetai
    return X

In [ ]:
eg.df['X'] = [X_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Z'] = [Z_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]


In [ ]:
eg.df['Q']=(eg.df['Z']-eg.df['theta0']*eg.df['X'])/eg.df['X']


In [ ]:
eg.df['grow'] = eg.df['y'].pct_change()

In [ ]:
eg.df

In [ ]:
eg.df.describe()

In [ ]:

def static_find(t):
    static=0
    N=0
    N_plus=0
    y=eg.df['y'][t]
    c=eg.df['c'][t]
    X=eg.df['X'][t]
    Q=eg.df['Q'][t]
    
    y_plus=eg.df['y'][t+1]
    c_plus=eg.df['c'][t+1]
    X_plus=eg.df['X'][t+1]
    Q_plus=eg.df['Q'][t+1]
    
    for g in range(0,n_goods):
               
        thetag='theta'+str(g)
        ag='a'+str(g)
               
        thetai=eg.df[thetag][t]
        theta0=eg.df['theta0'][t]
        xi=x_find(g,t)
        ai=eg.df[ag][t]
        a0=eg.df['a0'][t]
        
        thetai_plus=eg.df[thetag][t+1]
        theta0_plus=eg.df['theta0'][t+1]
        xi_plus=x_find(g,t+1)
        
        if g==0:
           si=eg.df['s0'][t]
           ni=eg.df['n0'][t]
           s0=si
           n0=ni
           
           
           ni_plus=eg.df['n0'][t+1]
           
           n0check=(1-theta0)*(1-c/y*(1-xi/X))/(1-theta0-c/y*Q)
           ni=n0check
        else:
            ni=xi/X*c/y*(1-thetai)/(1-theta0-c/y*Q)
            si=thetai/(1-thetai)*theta0/(1-theta0)*s0/n0
            
            ni_plus=xi_plus/X_plus*c_plus/y_plus*(1-thetai_plus)/(1-theta0_plus-c_plus/y_plus*Q_plus)
            
         
        k=eg.df['k'][t-1]
        yi=(si*k)**thetai*(ai*ni)**(1-thetai)
        static+=(ni_plus-ni)*yi
        N+=ni
        N_plus+=ni_plus
        
        print(g)
        if g==2:
            (print(N,N_plus))
            (print('check',n0,n0check))
    return static/y

In [ ]:
def py_n(tt,g):
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    a0=eg.df['a0'][tt]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    k=eg.df['k'][tt-1]
    prod=(1-theta0)*a0^(1-theta0)*k**theta0*(s0/n0)**theta0/(1-thetag)
    return prod

In [ ]:
def n_i(tt,g):
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    y=eg.df['y'][tt]
    c=eg.df['c'][tt]
    X=eg.df['X'][tt]
    Q=eg.df['Q'][tt]
    xi=x_find(g,tt)
    if g>0:
        ni=xi/X*c/y*(1-thetai)/(1-theta0-c/y*Q)
    else:
        ni=(1-theta0)*(1-c/y*(1-xi/X))/(1-theta0-c/y*Q)
    return ni

In [ ]:
def static_find(t):
    static=0
    N=0
    N_plus=0
    y=eg.df['y'][t]
    for g in range(0,n_goods):
        staticg=(n_i(t+1,g)-n_i(t,g))*py_n(t,g)
        print(g)
        N+=n_i(t,g)
        N_plus+=n_i(t+1,g)
        if g==2:
            (print('check1', N,N_plus))
        if g==0:
            (print('check2',n_i(t,g)-eg.df['n0'][t]))
        static+=staticg
    return static/y

In [ ]:
def within_find(t):
    within=0
    y=eg.df['y'][t]
    for g in range(0,n_goods):
        withing=(py_n(t+1,g)/n_i(t+1,g)-py_n(t,g)/n_i(t,g))*n_i(t,g)
    within+=withing
    return within/y

In [ ]:
def dynamic_find(t):
    dynamic=0
    y=eg.df['y'][t]
    for g in range(0,n_goods):
        dynamicg=(py_n(t+1,g)/n_i(t+1,g)-py_n(t,g)/n_i(t,g))*n_i(t,g)
    dynamic+=dynamicg
    return dynamic/y

In [ ]:
eg.df['static'] = [static_find(t) if t > 0 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]

In [ ]:
eg.df

In [ ]:
.001/.02


In [ ]:
eg.df.describe()

In [ ]:
n_goods

In [ ]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov